In [10]:
import mne
import numpy as np
import pandas as pd

from mne.time_frequency import tfr_array_morlet
from mne.baseline import rescale

from matplotlib import pyplot as plt
from mne.viz import centers_to_edges

In [11]:
# Use Qt backend for Matplotlib to plot in a new window
%matplotlib qt

In [12]:
# For creating dummy eeg time series data
# Define parameters
duration = 60  # 1 minute
sampling_frequency = 1000  # 1000 Hz
num_samples = duration * sampling_frequency

# Generate dummy EEG data (random values for simplicity)
eeg_data = np.random.randn(num_samples)

# Create a DataFrame
time = np.linspace(0, duration, num_samples, endpoint=False)
df = pd.DataFrame({'Time (s)': time, 'EEG Channel 1': eeg_data})

# Save to an Excel file
file_path = '/Users/jiontominaga/Research/mouse_eeg/data/dummy_eeg_data.xlsx'
df.to_excel(file_path, index=False)

file_path

'/Users/jiontominaga/Research/mouse_eeg/data/dummy_eeg_data.xlsx'

In [13]:
# For reshaping data to correct array shape
eeg_data_path = file_path
df = pd.read_excel(eeg_data_path)
# Extract the EEG data as a numpy array
eeg_data = df['EEG Channel 1'].values
# Extract the 'Time (s)' column as a numpy array
times_array = df['Time (s)'].values

# Reshape the data to [n_epochs, n_channels, n_times]
# Since it's a single time series data and a single channel, we set n_epochs to 1 and n_channels to 1
n_epochs = 1
n_channels = 1
n_times = eeg_data.shape[0]

# Reshape the EEG data
eeg_array = eeg_data.reshape((n_epochs, n_channels, n_times))

# Display the shape to verify
print(eeg_array.shape)

(1, 1, 60000)


In [14]:
# For calculating time-frequency representation
#1. set parameters for TFA:
min_freq = 0.1
max_freq = 30
step_freq = 1
sfreq = 1000
freqs = np.arange(min_freq, max_freq, step_freq)
n_cycles = freqs / 2.0
vmin, vmax = -3.0, 3.0  # Define our color limits.
power = tfr_array_morlet(eeg_array, sfreq=sfreq, freqs=freqs, n_cycles=n_cycles, output="avg_power")
# Baseline the output
rescale(power, times_array, (0.0, 0.1), mode="mean", copy=False)
fig, ax = plt.subplots()
x, y = centers_to_edges(times_array * 1000, freqs)
mesh = ax.pcolormesh(x, y, power[0], cmap="RdBu_r", vmin=vmin, vmax=vmax)
ax.set_title("TFR calculated on a numpy array")
ax.set(ylim=freqs[[0, -1]], xlabel="Time (ms)")
fig.colorbar(mesh)
plt.tight_layout()

plt.show()

Applying baseline correction (mode: mean)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
